In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go

import os

In [97]:
def read_data(fname, dir="../data/"):
    # Read data from csv file
    return pd.read_csv(os.path.join(dir, fname))

def rgba_to_hex(rgba):
    r, g, b, a = rgba
    if a == 1:
        return '#{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255))
    else:
        return '#{:02x}{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255), int(a*255))

In [98]:
jet_df_path = '../data/jet_df.csv'
particle_df_path = '../data/particle_df.csv'

jet_df = pd.read_csv(jet_df_path).set_index('jetID')
par_df = pd.read_csv(particle_df_path).set_index('jetID', append=True, drop=False)
preprocessed_df = par_df.reset_index(drop=True)

jet_df.head(10)

,eventID,nParticles,jetArea,jetPx,jetPy,jetPz,jetE,jetPolarPx,jetPolarPy,jetPolarPz,jetPolarE,jetPhi,jetTheta
jetID,,,,,,,,,,,,,
0,0,23,2.004635,-241.374752,0.754266,219.719568,330.240926,241.375931,0.816488,3.138468,50.200008,3.138468,0.832331
1,0,41,1.974715,178.037584,-5.547558,536.195118,566.155630,178.123993,1.821678,-0.031149,36.039368,-0.031149,0.320730
2,1,26,2.004635,157.606353,198.511810,-237.355351,348.306921,253.469330,-0.835698,0.899762,27.082243,0.899762,2.323376
3,1,20,2.024582,-141.065390,-96.179119,-883.349503,900.496654,170.733322,-2.345976,-2.543183,37.921139,-2.543183,2.950667
4,2,62,2.044528,56.939189,187.775177,-313.881867,372.725501,196.218216,-1.248802,1.276378,43.599136,1.276378,2.582897
5,2,35,1.994662,-6.843687,-176.672599,395.882462,434.952749,176.805099,1.545718,-1.609514,34.654400,-1.609514,0.420031
6,3,9,2.044528,-253.859634,12.243771,-564.509735,623.377403,254.154724,-1.538368,3.093400,73.030960,3.093400,2.718554
7,3,4,2.343728,221.186913,-62.356887,-144.815456,285.624219,229.808685,-0.594511,-0.274788,88.304283,-0.274788,2.133095
8,4,46,2.034555,-135.936301,272.007697,-199.728113,368.516492,304.083649,-0.616933,2.034245,58.704517,2.034245,2.151951


In [99]:
first_jet_part = par_df[par_df["jetID"]==0]
len(first_jet_part)

23

In [100]:
first_jet = jet_df[:1]
first_jet

,eventID,nParticles,jetArea,jetPx,jetPy,jetPz,jetE,jetPolarPx,jetPolarPy,jetPolarPz,jetPolarE,jetPhi,jetTheta
jetID,,,,,,,,,,,,,
0,0,23,2.004635,-241.374752,0.754266,219.719568,330.240926,241.375931,0.816488,3.138468,50.200008,3.138468,0.832331


In [101]:
first_jet_part.head()

,,eventID,jetID,particleType,particleVx,particleVy,particleVz,particlePx,particlePy,particlePz,particleE,particlePolarPx,particlePolarPy,particlePolarPz,particlePolarE,particlePhi,particleTheta
,jetID,,,,,,,,,,,,,,,,
0,0,0,0,0,0.000000,0.000000,0.000000,-115.595071,5.513218,107.093643,157.675996,115.726471,0.827630,3.093935,2.347607e-01,3.093935,0.824122
1,0,0,0,0,0.000000,0.000000,0.000000,-83.072377,4.831796,75.798599,112.561324,83.212776,0.816948,3.083494,5.078805e-01,3.083494,0.831991
2,0,0,0,-211,-0.981025,1.422285,-33.456345,-11.168506,-8.774579,9.043395,16.838385,14.203125,0.600055,-2.475661,1.395264e-01,-2.475661,1.003814
3,0,0,0,130,0.073932,0.089866,-2.399344,-8.233158,-1.087632,6.647210,10.637351,8.304688,0.732994,-3.010249,-1.192093e-07,-3.010249,0.895801
4,0,0,0,-211,0.073905,0.089409,-2.399101,-8.048296,0.478376,6.097900,10.109785,8.062500,0.698202,3.082224,1.395264e-01,3.082224,0.923257


The first step in the edge convolutional block is to find the k nearest neighbourn of each particle in each jet

Dobbiamo trovare i primi 10 vicini per ogni particella. Con essi ci servono i loro indici e le distanze relative con la prticella

In [130]:
def deltaR(eta1, phi1, eta2, phi2):
    delta_eta = eta1-eta2
    delta_phi = phi1-phi2

    return np.sqrt(delta_eta**2 + delta_phi**2)

# def inverse_deltaR(deltaR, eta_0, phi_0):
#     return 0

In [103]:
p_df = first_jet_part
idx = [i for i in range(len(p_df)) if i !=1]

Si può convertire in una funzione che prende in input il dataframe di un singolo jet e come output da una lista di liste di idx o di $\Delta R$ ecc..

In [200]:
eta = 'particlePolarPy'
phi = 'particlePhi'
k = 10
nn_idx_list  = []
nn_dist_list = []
nn_list = [] # list of pairs (idx, deltaR) of the k nn of the particle p

for p in range(len(p_df)):
    part  = p_df.iloc[p]
    eta_0 = part[eta]
    phi_0 = part[phi]

    idx   = [i for i in range(len(p_df)) if i !=p]
    other = p_df.iloc[idx]
    distances = [deltaR(eta_0, phi_0, other[eta].iloc[j], other[phi].iloc[j]) for j in range(len(other))]
    df = pd.DataFrame.from_dict({'distances':distances, 'eta':other[eta], 'phi':other[phi]})
    df = df.sort_values(by=['distances'])
    nn_df = df.head(k)

    nn_dist = np.array(nn_df.distances) #list of the distances of the k nn
    nn_idx  = [nn_df.index[i][0] for i in range(k)] #list of the indexes of the k nn

    nn_part_list = list(zip(nn_idx, nn_dist))
    nn_list.append(nn_part_list) 

len(nn_list)


3